In [1]:
from importlib import import_module
from pathlib import Path

from app.factories.context_provider import ContextProviderFactory
from app.factories.prompt_manager import PromptGeneratorFactory

import_module("app.extensions.context_providers")
import_module("app.extensions.prompt_generators")

sample = Path('sample_module.py')
sample.write_text('''
def foo(x):
    return x + 1

class Bar:
    def baz(self, y):
        return foo(y)
''')

provider = ContextProviderFactory.create('symbol_graph', module_path=str(sample))
context = provider.get_context()
print(context)

agent_cfg = {'template': 'Agent instructions'}
system_cfg = {'template': 'System instructions'}
generator = PromptGeneratorFactory.create('basic', context_provider=provider)
prompt = generator.generate_prompt(agent_cfg, system_cfg)
print(prompt)

assert 'foo(' in prompt
assert 'Agent instructions' in prompt
assert 'System instructions' in prompt
